In [1]:
import ccxt
import pandas as pd
import time
from datetime import datetime, timedelta
import numpy as np

In [4]:
class Trade:
    def __init__(self, entry_time, entry_price, trade_type, position_type, cash, size=1, commission_rate=0):
        self.entry_time = entry_time
        self.entry_price = entry_price
        self.trade_type = trade_type
        self.position_type = position_type
        self.size = size
        self.exit_time = None
        self.exit_price = None
        self.exit_type = None
        self.in_pos = None
        self.profit = 0
        self.pr_profit = 0
        self.commission_rate = commission_rate
        self.cash = cash

    def close(self, exit_time, exit_price, exit_type):
        self.exit_time = exit_time
        self.exit_price = exit_price
        self.exit_type = exit_type
        if self.trade_type == 'long':
            self.pr_profit = ((
                                      self.exit_price - self.entry_price) / self.entry_price - self.commission_rate) * self.size
        elif self.trade_type == 'short':
            self.pr_profit = ((
                                      self.entry_price - self.exit_price) / self.entry_price - self.commission_rate) * self.size
        self.in_pos = (self.exit_time - self.entry_time).total_seconds() / 86400
        return self.pr_profit

In [8]:
colors = ['black', 'blue', 'brown', 'green', 'orange', 'red', 'violet', 'yellow']


# Functions
def log_open(trade, position_type, n_date):
    date = trade.entry_time
    type = trade.trade_type
    entry = trade.entry_price

    print({'date': date, 'position': position_type, 'type': type, 'entry': entry, 'n_date': n_date})


def log_close(trade, n_date):
    date = trade.exit_time
    type = trade.trade_type
    exit = trade.exit_price
    exit_type = trade.exit_type
    position_type = trade.position_type

    print(
        {'date': date, 'position': position_type, 'type': type, 'exit': exit, 'n_date': n_date, 'exit type': exit_type})


def fetch_data(exchange, symbol, since, limit=1000):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1h', since=since, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('Date', inplace=True)
    return df.ffill()


def Read_Currency(exchange, symbol_name, hours):
    since = exchange.parse8601((datetime.utcnow() - timedelta(hours=hours)).strftime('%Y-%m-%dT%H:%M:%S'))
    df = fetch_data(exchange, symbol_name, since)
    return df


# find trend output is from 0 to 6.
def FindTrend(data, current_index, previous_index):
    c_array = [data.iloc[current_index].loc['C-Open'], data.iloc[current_index].loc['C-Close'],
               data.iloc[current_index].loc['D-Open'], data.iloc[current_index].loc['D-Close']]
    p_array = [data.iloc[previous_index].loc['C-Open'], data.iloc[previous_index].loc['C-Close'],
               data.iloc[previous_index].loc['D-Open'], data.iloc[previous_index].loc['D-Close']]
    Cmaxt = max(c_array)
    Cmint = min(c_array)
    Cmaxp = max(p_array)
    Cminp = min(p_array)

    Candles_min = min([Cminp, Cmint])
    Candles_max = max([Cmaxp, Cmaxt])

    min_diff = []
    max_diff = []

    a = Cmaxp - Cminp
    b = Cmaxt - Cmint
    if a == 0:
        min_diff.append(1)
        max_diff.append(1)
    else:
        min_diff.append(abs(Cminp - Candles_min) / (Cmaxp - Cminp))
        max_diff.append(abs(Candles_max - Cmaxp) / (Cmaxp - Cminp))

    if b == 0:
        min_diff.append(1)
        max_diff.append(1)
    else:
        min_diff.append(abs(Cmint - Candles_min) / (Cmaxt - Cmint))
        max_diff.append(abs(Candles_max - Cmaxt) / (Cmaxt - Cmint))

    C_meant = (Cmaxt + Cmint) / 2
    C_meanp = (Cmaxp + Cminp) / 2

    if sum(min_diff) <= 0.381 and sum(max_diff) <= 0.381:  # type 3
        return 3
    if sum(min_diff) > 0.381 and sum(max_diff) > 0.381:
        if (Cminp > Cmint) and (Cmaxp < Cmaxt) and \
                (Cminp < C_meant) and (Cmaxp > C_meant):
            return 3
        elif (Cminp < Cmint) and (Cmaxp > Cmaxt) and \
                (Cmint < C_meanp) and (Cmaxt > C_meanp):
            return 3

    if sum(max_diff) <= 0.381:  # type 1 & 4
        if (Cminp < Cmint) and (C_meanp <= Cmaxt) and (C_meant <= Cmaxp):
            return 4
        elif (Cminp > Cmint) and (C_meant <= Cmaxp) and (C_meanp <= Cmaxt):
            return 1

    if sum(min_diff) <= 0.381:  # type 2 & 5
        if (Cmaxp < Cmaxt) and (C_meanp >= Cmint) and (C_meanp >= Cmint):
            return 5
        elif (Cmaxp > Cmaxt) and (C_meant >= Cminp) and (C_meant >= Cminp):
            return 2

    if Cmaxp < Cmaxt and Cminp >= C_meant:
        return 1
    if Cmaxt < Cmaxp and Cmint >= C_meanp:
        return 4
    if Cminp > Cmint and Cmaxp <= C_meant:
        return 5
    if Cmint > Cminp and Cmaxt <= C_meanp:
        return 2
    if C_meanp > C_meant:
        return 0

    if C_meanp < C_meant:
        return 6

    return 7


#---------------------------------------------------------
# trend
# -1: none. 0: downtrend. 1: uptrend
def TrendCheck(data, current_index, previous_index):
    c_array = [data.iloc[current_index].loc['C-Open'], data.iloc[current_index].loc['C-Close'],
               data.iloc[current_index].loc['D-Open'], data.iloc[current_index].loc['D-Close']]
    p_array = [data.iloc[previous_index].loc['C-Open'], data.iloc[previous_index].loc['C-Close'],
               data.iloc[previous_index].loc['D-Open'], data.iloc[previous_index].loc['D-Close']]
    up = 0
    down = 0
    if data.iloc[current_index].loc['T-Mean'] > data.iloc[previous_index].loc['T-Mean']:
        cMax1 = max(c_array)
        cMax2 = max(p_array)
        if cMax1 >= cMax2:
            up = 1
    if data.iloc[current_index].loc['T-Mean'] < data.iloc[previous_index].loc['T-Mean']:
        cMin1 = min(c_array)
        cMin2 = min(p_array)
        if cMin1 <= cMin2:
            down = 1

    if up + down == 2:
        return -1
    elif up == 1:
        return 1
    elif down == 1:
        return 0

    return -1


#--------------------------------------
# candle state from 1 to 6
def CandleState(data, index):
    array1 = [data.iloc[index].loc['C-Open'], data.iloc[index].loc['C-Close'], data.iloc[index].loc['D-Open'],
              data.iloc[index].loc['D-Close']]
    array2 = [data.iloc[index - 1].loc['C-Open'], data.iloc[index - 1].loc['C-Close'],
              data.iloc[index - 1].loc['D-Open'], data.iloc[index - 1].loc['D-Close']]
    array3 = [data.iloc[index - 2].loc['C-Open'], data.iloc[index - 2].loc['C-Close'],
              data.iloc[index - 2].loc['D-Open'], data.iloc[index - 2].loc['D-Close']]
    up = 0
    down = 0
    if max(array1) > max(array2) and \
            max(array1) > max(array3):
        up += 1
    if min(array1) < min(array2) and \
            min(array1) < min(array3):
        down += 1
    if up + down == 2:
        return 1  # both up and down
    elif FindTrend(data, index, index - 1) in [4, 5, 6] and FindTrend(data, index, index - 2) in [5, 6]:
        return 2  # upper
    elif FindTrend(data, index, index - 1) in [0, 1, 2] and FindTrend(data, index, index - 2) in [0, 1]:
        return 3  # lower
    elif data.iloc[index].loc['T-Mean'] >= max(array2):
        return 4  # down-up
    elif data.iloc[index].loc['T-Mean'] <= min(array2):
        return 5  # up-down
    else:
        return 6  # O.W.


#----------------------------------------------------------------------
# Check colors

# get signal color type
def SignalType(index, alpha, beta, data):
    cT = -1
    cP = -1
    for k in range(3, index):
        cT = TrendCheck(data, index, index - k)
        if cT != -1:
            break
    for k in range(3, index - 1):
        cP = TrendCheck(data, index - 1, index - 1 - k)
        if cP != -1:
            break
    if data.iloc[index].loc['vrate'] > alpha:
        if cP == 0 and cT == 0:
            return 'brown'
        elif cP == 1 and cT == 0:
            return 'orange'
        elif cP == 1 and cT == 1:
            return 'blue'
        else:
            return 'violet'
    elif data.iloc[index].loc['vrate'] < beta:
        if cP == 0 and cT == 0:
            return 'black'
        elif cP == 1 and cT == 0:
            return 'yellow'
        elif cP == 1 and cT == 1:
            return 'green'
        else:
            return 'red'

    return 'none'


# Mojesa Func
def Mojesa_Class_Func(ohcl_data):
    new_df = {}

    # date
    new_df['Date'] = ohcl_data.index[0]

    # real date
    new_df['O-Open'] = ohcl_data.iloc[0].loc['Open']
    new_df['O-Close'] = ohcl_data.iloc[-1].loc['Close']
    new_df['O-High'] = max(ohcl_data['High'])
    new_df['O-Low'] = min(ohcl_data['Low'])
    new_df['O-Volume'] = sum(ohcl_data['Volume'])

    if (new_df['O-Open'] >= new_df['O-Close']):
        bullType = 'black'
    else:
        bullType = 'white'
    new_df['O-Bull'] = bullType

    # get order of data candles
    # find orders
    C_Height1 = abs(ohcl_data.iloc[0].loc['Close'] - ohcl_data.iloc[0].loc['Open'])
    C_Height2 = abs(ohcl_data.iloc[1].loc['Close'] - ohcl_data.iloc[1].loc['Open'])
    C_Height3 = abs(ohcl_data.iloc[2].loc['Close'] - ohcl_data.iloc[2].loc['Open'])

    order = np.array(ohcl_data['Volume'].rank(method='max', ascending=False)).flatten()

    if order[0] == order[1] and order[0] == order[2]:
        if C_Height3 < C_Height1 and C_Height3 < C_Height2:
            order[2] = 1
            if C_Height2 < C_Height1:
                order[1] = 2
                order[0] = 3
            else:
                order[1] = 3
                order[0] = 2
        elif C_Height2 < C_Height1:
            order[1] = 1
            if C_Height3 < C_Height1:
                order[2] = 2
                order[0] = 3
            else:
                order[2] = 3
                order[0] = 2
        else:
            order[0] = 1
            if C_Height3 < C_Height2:
                order[2] = 2
                order[1] = 3
            else:
                order[2] = 3
                order[1] = 2
    elif order[0] == order[1]:
        if C_Height2 < C_Height1:
            order[1] -= 1
        else:
            order[0] -= 1
    elif order[0] == order[2]:
        if C_Height3 < C_Height1:
            order[2] -= 1
        else:
            order[0] -= 1
    elif order[1] == order[2]:
        if C_Height3 < C_Height2:
            order[2] -= 1
        else:
            order[1] -= 1

    # ohcl_data['Order'] = order

    #-----------------------------------------------------
    # divide data based on orders
    data1 = []
    data2 = []
    data3 = []

    for i in range(0, ohcl_data.shape[0]):
        match order[i]:
            case 1:
                data1 = ohcl_data.iloc[i].copy()
            case 2:
                data2 = ohcl_data.iloc[i].copy()
            case 3:
                data3 = ohcl_data.iloc[i].copy()

    #-----------------------------------------------------
    # correct devided date time and add bull parameter to them

    Date = ohcl_data.index[0]
    data1.name = Date
    data2.name = Date
    data3.name = Date

    bullType = 'none'
    if (data1['Open'] >= data1['Close']):
        bullType = 'black'
    else:
        bullType = 'white'
    data1['Bull'] = bullType

    bullType = 'none'
    if (data2['Open'] >= data2['Close']):
        bullType = 'black'
    else:
        bullType = 'white'
    data2['Bull'] = bullType

    bullType = 'none'
    if (data3['Open'] >= data3['Close']):
        bullType = 'black'
    else:
        bullType = 'white'
    data3['Bull'] = bullType

    #---------------------------------------------------------
    # calculate combinedData (black/white candle)

    combinedData = data1.copy()

    a = abs(data1['Open'] - data1['Close'])
    b = abs(data2['Open'] - data2['Close'])
    maxindex = pd.Series([a, b]).idxmax()

    openmax = max([data1['Open'], data2['Open']])
    openmin = min([data1['Open'], data2['Open']])

    closemax = max([data1['Close'], data2['Close']])
    closemin = min([data1['Close'], data2['Close']])

    match maxindex:
        case 0:
            if data1['Bull'] == 'black':
                combinedData['Open'] = max([openmax, closemax])
                combinedData['Close'] = min([openmin, closemin])
            else:
                combinedData['Open'] = min([openmin, closemin])
                combinedData['Close'] = max([openmax, closemax])
        case 1:
            if data2['Bull'] == 'black':
                combinedData['Open'] = max([openmax, closemax])
                combinedData['Close'] = min([openmin, closemin])
            else:
                combinedData['Open'] = min([openmin, closemin])
                combinedData['Close'] = max([openmax, closemax])

    combinedData['High'] = max([data1['High'], data2['High']])
    combinedData['Low'] = min([data1['Low'], data2['Low']])
    combinedData['Volume'] = sum([data1['Volume'], data2['Volume']])

    if combinedData['Open'] >= combinedData['Close']:
        combinedData['Bull'] = 'black'
    else:
        combinedData['Bull'] = 'white'

    new_df['C-Open'] = combinedData['Open']
    new_df['C-High'] = combinedData['High']
    new_df['C-Close'] = combinedData['Close']
    new_df['C-Low'] = combinedData['Low']
    new_df['C-Volume'] = combinedData['Volume']
    new_df['C-Bull'] = combinedData['Bull']

    new_df['D-Open'] = data3['Open']
    new_df['D-High'] = data3['High']
    new_df['D-Close'] = data3['Close']
    new_df['D-Low'] = data3['Low']
    new_df['D-Volume'] = data3['Volume']
    new_df['D-Bull'] = data3['Bull']

    new_df['T-Open'] = min([combinedData['Open'], combinedData['Close'], data3['Open'], data3['Close']])
    new_df['T-High'] = new_df['O-High']
    new_df['T-Close'] = max([combinedData['Open'], combinedData['Close'], data3['Open'], data3['Close']])
    new_df['T-Low'] = new_df['O-Low']

    #--------------------------------------------------------------------------
    # calculate data data parameters

    Co_mean = (combinedData['Open'] + combinedData['Close']) / 2.0
    D_mean = (data3['Open'] + data3['Close']) / 2.0
    C_mean = (new_df['T-Open'] + new_df['T-Close']) / 2.0

    body_compare = 0
    x1 = abs(combinedData['Close'] - combinedData['Open'])
    x2 = abs(data3['Close'] - data3['Open'])
    if x2 == x1:
        body_compare = 1
    elif x2 == 0:
        body_compare = 1000
    else:
        body_compare = x1 / x2

    vrate = 0
    if combinedData['Volume'] == 0:
        vrate = 1000
    else:
        vrate = (data3['Volume']) / combinedData['Volume']
    # stRSI , stRSI_K , stRSI_D = StochRSI(combinedData.Close)

    new_df['T-Mean'] = C_mean
    new_df['C-Mean'] = Co_mean
    new_df['D-Mean'] = D_mean
    new_df['brate'] = body_compare
    new_df['vrate'] = vrate

    return new_df


def wait_until_divisible_by_3():
    while True:
        utc_now = datetime.utcnow()
        if utc_now.hour % 3 == 0 and utc_now.minute == 0:
            print(f"Current UTC time {utc_now} is divisible by 3. Starting the process.")
            break
        else:
            print(f"Waiting... Current UTC time: {utc_now}")
        time.sleep(60)


def add_t_s_c(sym_df):
    C_Trend = []
    C_State = []
    sigcolor = []
    for i in range(0, sym_df.shape[0]):
        if i < 2:
            C_Trend.append({'Date': sym_df.index[i], 'Trend': np.nan})
            C_State.append({'Date': sym_df.index[i], 'State': np.nan})
            sigcolor.append({'Date': sym_df.index[i], 'color': np.nan})
        else:
            C_Trend.append({'Date': sym_df.index[i], 'Trend': FindTrend(sym_df, i, i - 1)})
            C_State.append({'Date': sym_df.index[i], 'State': CandleState(sym_df, i)})
            sigcolor.append({'Date': sym_df.index[i], 'color': SignalType(i, 0.39, 0.2, sym_df)})

    C_Trend = pd.DataFrame(C_Trend)
    C_Trend.set_index('Date', inplace=True)
    C_State = pd.DataFrame(C_State)
    C_State.set_index('Date', inplace=True)
    sigcolor = pd.DataFrame(sigcolor)
    sigcolor.set_index('Date', inplace=True)

    sym_df['Trend'] = C_Trend
    sym_df['State'] = C_State
    sym_df['Color'] = sigcolor

    return sym_df


def check_open_conditions(df):
    df = df.reindex()
    for index, row in df.iterrows():
        l = len(df)

        dattee = row['Date']
        color = row['Color']
        state = row['State']
        trend = row['Trend']
        bul = row['C-Bull']

        t_close = row['T-Close']
        t_open = row['T-Open']
        t_low = row['T-Low']
        t_high = row['T-High']

        o_close = row['O-Close']
        o_open = row['O-Open']
        o_low = row['O-Low']
        o_high = row['O-High']

        b_color = 'none'
        a_color = 'none'
        b_index = -1
        a_index = -1

        try:
            for i in range(index - 1, max(index - 5, 0) - 1, -1):
                c_color = df.loc[i, 'Color']
                if c_color in colors:
                    b_color = c_color
                    b_index = i
                    break

            for i in range(index + 1, index + 6):
                c_color = df.loc[i, 'Color']
                if c_color in colors:
                    a_color = c_color
                    a_index = i
                    break
        except:
            pass

        if a_index == -1:
            a_index = min(index + 5, l - 1)
            a_color = 'none'
            a_bul = df.loc[a_index, 'C-Bull']
            a_brate = df.loc[a_index, 'brate']
        else:
            a_bul = df.loc[a_index, 'C-Bull']
            a_brate = df.loc[a_index, 'brate']

        if b_index == -1:
            b_index = max(index - 5, 0)
            b_color = 'none'
            b_bul = df.loc[b_index, 'C-Bull']
            b_brate = df.loc[b_index, 'brate']
        else:
            b_bul = df.loc[b_index, 'C-Bull']
            b_brate = df.loc[b_index, 'brate']

        try:
            next_bul = df.loc[index + 1, 'C-Bull']
        except:
            return ['nothing', None]
        try:
            next_a_bul = df.loc[a_index + 1, 'C-Bull']
        except:
            next_a_bul = None

        try:
            if color == 'black' and state == 3 and trend == 0:
                if b_color == 'black' and a_color == 'black':
                    #**1
                    if bul == 'white':
                        return ['open', [df.index[a_index], 'black.black3.0.black.1', 'long', -1]]
                        # approved_candle = a_index
                        # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.1'
                        # df.loc[approved_candle, 'signal type'] = 'long'
                        # df.loc[approved_candle, 'buy price'] = -1
                        # continue
                    #**2
                    if bul == 'black' and b_bul == 'white':
                        #*1
                        if a_index != index + 1 and next_bul == 'white':
                            #1
                            if b_brate <= 1 or b_brate >= 4.2:
                                return ['open', [df.index[a_index], 'black.black3.0.black.2.1.1', 'long', -1]]
                                # approved_candle = a_index
                                # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.1.1'
                                # df.loc[approved_candle, 'signal type'] = 'long'
                                # df.loc[approved_candle, 'buy price'] = -1
                                # continue
                            #2
                            for i in range(index + 1, a_index):
                                candle_bul = df.loc[i, 'C-Bull']
                                if candle_bul == 'black':
                                    return ['open', [df.index[a_index], 'black.black3.0.black.2.1.2', 'short', -1]]
                                    # approved_candle = a_index
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.1.2'
                                    # df.loc[approved_candle, 'signal type'] = 'short'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # continue

                            #3
                            if next_a_bul == 'white':
                                return ['open', [df.index[a_index], 'black.black3.0.black.2.1.3', 'long', -1]]
                                # approved_candle = a_index
                                # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.1.3'
                                # df.loc[approved_candle, 'signal type'] = 'long'
                                # df.loc[approved_candle, 'buy price'] = -1
                                # continue
                            else:
                                return ['open', [df.index[a_index], 'black.black3.0.black.2.1.3', 'short', -1]]
                                # approved_candle = a_index
                                # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.1.3'
                                # df.loc[approved_candle, 'signal type'] = 'short'
                                # df.loc[approved_candle, 'buy price'] = -1
                                # continue
                        #*2
                        if a_index == index + 1 and next_bul == 'white':
                            #1
                            if b_brate <= 1 or b_brate == 1000:
                                return ['open', [df.index[a_index], 'black.black3.0.black.2.2.1', 'long', -1]]
                                # approved_candle = a_index
                                # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.2.1'
                                # df.loc[approved_candle, 'signal type'] = 'long'
                                # df.loc[approved_candle, 'buy price'] = -1
                                # continue
                            #2
                            if next_a_bul == 'white':
                                return ['open', [df.index[a_index + 1], 'black.black3.0.black.2.2.2', 'short', -1]]
                                # approved_candle = a_index + 1
                                # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.2.2'
                                # df.loc[approved_candle, 'signal type'] = 'short'
                                # df.loc[approved_candle, 'buy price'] = -1
                                # continue
                            #3
                            if next_a_bul == 'black':
                                next_next_a_bul = df.loc[a_index + 2, 'C-Bull']
                                if next_next_a_bul == 'black':
                                    return ['open', [df.index[a_index + 2], 'black.black3.0.black.2.2.3', 'short', -1]]
                                    # approved_candle = a_index + 2
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.2.3'
                                    # df.loc[approved_candle, 'signal type'] = 'short'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # continue
                                else:
                                    return ['open', [df.index[a_index + 2], 'black.black3.0.black.2.2.3', 'long', -1]]
                                    # approved_candle = a_index + 2
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.2.3'
                                    # df.loc[approved_candle, 'signal type'] = 'long'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # continue
                        #*3
                        if next_bul == 'black':
                            flag = 0
                            #1
                            for i in range(index + 1, a_index):
                                candle_bul = df.loc[i, 'C-Bull']
                                if candle_bul == 'white':
                                    return ['open', [df.index[a_index], 'black.black3.0.black.2.3.1', 'short', -1]]
                                    # approved_candle = a_index
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.1'
                                    # df.loc[approved_candle, 'signal type'] = 'short'
                                    # df.loc[approved_candle, 'buy price'] = -1

                            #2
                            for i in range(index + 1, a_index):
                                candle_bul = df.loc[i, 'C-Bull']
                                candle_brate = df.loc[i, 'brate']
                                if candle_bul == 'black' and candle_brate <= 1.5:
                                    return ['open', [df.index[a_index], 'black.black3.0.black.2.3.2', 'long', -1]]
                                    # approved_candle = a_index
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.2'
                                    # df.loc[approved_candle, 'signal type'] = 'long'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # flag = 1
                                else:
                                    return ['open', [df.index[a_index], 'black.black3.0.black.2.3.2', 'short', -1]]
                                    # approved_candle = a_index
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.2'
                                    # df.loc[approved_candle, 'signal type'] = 'short'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # flag = 1

                            # if flag == 1:
                            #     continue

                            #3
                            next_a_cbody_low = min(df.loc[a_index + 1, 'O-Close'], df.loc[a_index + 1, 'O-Open'])
                            a_cbody_low = min(df.loc[a_index, 'O-Close'], df.loc[a_index, 'O-Open'])
                            if next_a_bul == 'white':
                                next_a_candle_lenght = abs(
                                    df.loc[a_index + 1, 'T-Close'] - df.loc[a_index + 1, 'T-Open'])
                                third_candle_lenght = abs(df.loc[a_index, 'T-Close'] - df.loc[a_index, 'T-Open'])
                                rate = third_candle_lenght / next_a_candle_lenght
                                if rate >= 3:
                                    return ['open', [df.index[a_index + 1], 'black.black3.0.black.2.3.3', 'short', -1]]
                                    # approved_candle = a_index + 1
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.3'
                                    # df.loc[approved_candle, 'signal type'] = 'short'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # continue
                                elif rate <= 1 and next_a_cbody_low <= a_cbody_low:
                                    return ['open', [df.index[a_index + 1], 'black.black3.0.black.2.3.3', 'short', -1]]
                                    # approved_candle = a_index + 1
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.3'
                                    # df.loc[approved_candle, 'signal type'] = 'short'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # continue
                                else:
                                    return ['open', [df.index[a_index + 1], 'black.black3.0.black.2.3.3', 'long', -1]]
                                    # approved_candle = a_index + 1
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.3'
                                    # df.loc[approved_candle, 'signal type'] = 'long'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # continue
                            #4
                            if next_a_bul == 'black':
                                flag = 0
                                for i in range(a_index + 1, a_index + 6):
                                    candle_bul = df.loc[i, 'C-Bull']
                                    if candle_bul == 'white':
                                        return ['open', [df.index[i], 'black.black3.0.black.2.3.4', 'long', -1]]
                                        # approved_candle = i
                                        # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.4'
                                        # df.loc[approved_candle, 'signal type'] = 'long'
                                        # df.loc[approved_candle, 'buy price'] = -1
                                        # flag = 1
                                        # break
                                # if flag == 1:
                                #     continue
                    #**3
                    if bul == 'black' and b_bul == 'black':
                        #*1
                        if a_index != index + 1 and next_bul == 'white':
                            #1
                            if a_bul == 'white':
                                #1.1
                                if a_brate <= 1:
                                    return ['open', [df.index[a_index], 'black.black3.0.black.3.1.1_1', 'long', -1]]
                                    # approved_candle = a_index
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.1_1'
                                    # df.loc[approved_candle, 'signal type'] = 'long'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # continue
                                #1.2
                                next_candle_lenght = abs(df.loc[index + 1, 'T-Close'] - df.loc[index + 1, 'T-Open'])
                                third_candle_lenght = abs(df.loc[a_index, 'T-Close'] - df.loc[a_index, 'T-Open'])
                                rate = next_candle_lenght / third_candle_lenght
                                if rate <= 1:
                                    return ['open', [df.index[a_index], 'black.black3.0.black.3.1.1_2', 'long', -1]]
                                    # approved_candle = a_index
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.1_2'
                                    # df.loc[approved_candle, 'signal type'] = 'long'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # continue
                                else:
                                    return ['open', [df.index[a_index], 'black.black3.0.black.3.1.1_2', 'short', -1]]
                                    # approved_candle = a_index
                                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.1_2'
                                    # df.loc[approved_candle, 'signal type'] = 'short'
                                    # df.loc[approved_candle, 'buy price'] = -1
                                    # continue
                            #2
                            if a_bul == 'black':
                                #2.1
                                flag = 0
                                for i in range(index + 1, a_index):
                                    candle_bul = df.loc[i, 'C-Bull']
                                    candle_brate = df.loc[i, 'brate']
                                    if candle_bul == 'white' and candle_brate <= 1:
                                        return ['open',
                                                [df.index[a_index], 'black.black3.0.black.3.1.2_1', 'short', -1]]
                                        # approved_candle = a_index
                                        # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_1'
                                        # df.loc[approved_candle, 'signal type'] = 'short'
                                        # df.loc[approved_candle, 'buy price'] = -1
                                        # flag = 1
                                        # break
                                # if flag == 1:
                                #     continue

                                #2.2
                                # next_a_body_mean = (df.loc[a_index + 1, 'T-Close'] + df.loc[a_index + 1, 'T-Open']) / 2
                                next_a_body_high = max(df.loc[a_index + 1, 'T-Close'], df.loc[a_index + 1, 'T-Open'])
                                next_next_a_body_high = max(df.loc[a_index + 2, 'T-Close'],
                                                            df.loc[a_index + 2, 'T-Open'])
                                if next_a_bul == 'white':
                                    candle_trend = df.loc[a_index + 1, 'Trend']
                                    if candle_trend <= 2:
                                        return ['open',
                                                [df.index[a_index + 1], 'black.black3.0.black.3.1.2_2', 'long', -1]]
                                        # approved_candle = a_index + 1
                                        # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_2'
                                        # df.loc[approved_candle, 'signal type'] = 'long'
                                        # df.loc[approved_candle, 'buy price'] = -1
                                        # continue
                                    elif candle_trend == 3:
                                        next_next_a_bul = df.loc[a_index + 2, 'C-Bull']
                                        if next_next_a_bul == 'white' and next_next_a_body_high > next_a_body_high:
                                            return ['open',
                                                    [df.index[a_index + 2], 'black.black3.0.black.3.1.2_2', 'long', -1]]
                                            # approved_candle = a_index + 2
                                            # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_2'
                                            # df.loc[approved_candle, 'signal type'] = 'long'
                                            # df.loc[approved_candle, 'buy price'] = -1
                                            # continue
                                        else:
                                            return ['open',
                                                    [df.index[a_index + 2], 'black.black3.0.black.3.1.2_2', 'short',
                                                     -1]]
                                            # approved_candle = a_index + 2
                                            # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_2'
                                            # df.loc[approved_candle, 'signal type'] = 'short'
                                            # df.loc[approved_candle, 'buy price'] = -1
                                            # continue
                                    else:
                                        next_next_a_bul = df.loc[a_index + 2, 'C-Bull']
                                        candle_trend = df.loc[a_index + 2, 'Trend']
                                        if next_next_a_bul == 'white' and candle_trend not in [5, 6]:
                                            return ['open',
                                                    [df.index[a_index + 2], 'black.black3.0.black.3.1.2_2', 'long', -1]]
                                            # approved_candle = a_index + 2
                                            # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_2'
                                            # df.loc[approved_candle, 'signal type'] = 'long'
                                            # df.loc[approved_candle, 'buy price'] = -1
                                            # continue
                                        else:
                                            return ['open',
                                                    [df.index[a_index + 2], 'black.black3.0.black.3.1.2_2', 'short',
                                                     -1]]
                                            # approved_candle = a_index + 2
                                            # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_2'
                                            # df.loc[approved_candle, 'signal type'] = 'short'
                                            # df.loc[approved_candle, 'buy price'] = -1
                                            # continue

                                #2.3
                                next_a_body_mean = (df.loc[a_index + 1, 'T-Close'] + df.loc[a_index + 1, 'T-Open']) / 2
                                body_mean = (df.loc[index, 'T-Close'] + df.loc[index, 'T-Open']) / 2
                                body_low = min(df.loc[index, 'T-Close'], df.loc[index, 'T-Open'])
                                if next_a_bul == 'black':
                                    if body_low < next_a_body_mean < body_mean:
                                        return ['open',
                                                [df.index[a_index + 1], 'black.black3.0.black.3.1.2_3', 'long', -1]]
                                        # approved_candle = a_index + 1
                                        # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_3'
                                        # df.loc[approved_candle, 'signal type'] = 'long'
                                        # df.loc[approved_candle, 'buy price'] = -1
                                        # continue
                                    else:
                                        return ['open',
                                                [df.index[a_index + 1], 'black.black3.0.black.3.1.2_3', 'short', -1]]
                                        # approved_candle = a_index + 1
                                        # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_3'
                                        # df.loc[approved_candle, 'signal type'] = 'short'
                                        # df.loc[approved_candle, 'buy price'] = -1
                                        # continue

                        #*2
                        next_a_body_mean = (df.loc[a_index + 1, 'T-Close'] + df.loc[a_index + 1, 'T-Open']) / 2
                        a_body_high = max(df.loc[a_index, 'T-Close'], df.loc[a_index, 'T-Open'])
                        next_a_candle_lenght = abs(df.loc[a_index + 1, 'T-Close'] - df.loc[a_index + 1, 'T-Open'])
                        third_candle_lenght = abs(df.loc[a_index, 'T-Close'] - df.loc[a_index, 'T-Open'])
                        rate = third_candle_lenght / next_a_candle_lenght
                        if a_index == index + 1 and next_bul == 'white':
                            #1
                            if next_a_bul == 'white':
                                if next_a_body_mean >= a_body_high:
                                    if rate <= 1:
                                        return ['open',
                                                [df.index[a_index + 1], 'black.black3.0.black.3.2.1', 'short', -1]]
                                        # approved_candle = a_index + 1
                                        # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.2.1'
                                        # df.loc[approved_candle, 'signal type'] = 'short'
                                        # df.loc[approved_candle, 'buy price'] = -1
                                        # continue
                                    else:
                                        return ['open',
                                                [df.index[a_index + 1], 'black.black3.0.black.3.2.1', 'long', -1]]
                                        # approved_candle = a_index + 1
                                        # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.2.1'
                                        # df.loc[approved_candle, 'signal type'] = 'long'
                                        # df.loc[approved_candle, 'buy price'] = -1
                                        # continue
                                #2
                                else:
                                    flag = 0
                                    for i in range(a_index + 2, a_index + 7):
                                        candle_bul = df.loc[i, 'C-Bull']
                                        candle_brate = df.loc[i, 'brate']
                                        if candle_bul == 'black':
                                            if candle_brate > 1 and candle_brate <= 15:
                                                return ['open',
                                                        [df.index[i], 'black.black3.0.black.3.2.2', 'short', -1]]
                                                # approved_candle = i
                                                # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.2.2'
                                                # df.loc[approved_candle, 'signal type'] = 'short'
                                                # df.loc[approved_candle, 'buy price'] = -1
                                                # flag = 1
                                                # break
                                            else:
                                                return ['open', [df.index[i], 'black.black3.0.black.3.2.2', 'long', -1]]
                                                # approved_candle = i
                                                # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.2.2'
                                                # df.loc[approved_candle, 'signal type'] = 'long'
                                                # df.loc[approved_candle, 'buy price'] = -1
                                                # flag = 1
                                                # break
                                    # if flag == 1:
                                    #     continue

                            #3
                            if next_a_bul == 'black':
                                return ['open', [df.index[a_index + 1], 'black.black3.0.black.3.2.3', 'long', -1]]
                                # approved_candle = a_index + 1
                                # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.2.3'
                                # df.loc[approved_candle, 'signal type'] = 'long'
                                # df.loc[approved_candle, 'buy price'] = -1
                                # continue

                        #*3
                        #?
        except:
            print("more candles")

    return ['nothing', None]


def check_close_condition(current_trade, df, stop_loss, current_price):
    row = df.iloc[-1]
    if current_trade.trade_type == 'long':
        if row['Color'] == 'blue' or row['Color'] == 'green':
            profit_percentage = (current_price - current_trade.entry_price) / current_trade.entry_price
            if profit_percentage > 0.01:
                exit_time = datetime.utcnow()
                current_trade.close(exit_time, current_price, 'blue-green')
                log_close(current_trade, row.name)
                return 'closed'

        stop_loss_price = current_trade.entry_price * (1 - stop_loss)
        if current_price <= stop_loss_price:
            profit = current_trade.close(row.name, stop_loss_price, 'stop loss')
            log_close('STOP LOSS HIT', stop_loss_price, row.name,
                      f"Exited long position due to stop loss at {stop_loss_price} on {row.name}")
            current_trade = None
            return

    elif current_trade.trade_type == 'short':
        if row['Low'] <= current_trade.entry_price * 0.99:
            profit = current_trade.close(row.name, current_trade.entry_price * 0.99, '1pr exit')
            log_close('BUY', row['Close'], row.name, f"Exited short position with 1% profit on {row.name}")
            current_trade = None
            return

        stop_loss_price = current_trade.entry_price * (1 + stop_loss)
        if row['High'] >= stop_loss_price:
            profit = current_trade.close(row.name, stop_loss_price, 'stop loss')
            log_close('STOP LOSS HIT', stop_loss_price, row.name,
                      f"Exited short position due to stop loss at {stop_loss_price} on {row.name}")
            current_trade = None
            return


def find_last_three_colors(df):
    colors_seen = []

    for i in reversed(df.index):
        color = df.at[i, 'color']

        if color == 'none':
            colors_seen.append(color)

        if len(colors_seen) == 3:
            return df.iloc[i:]

    return None

In [9]:
exchange = ccxt.mexc()
symbol = 'BTC/USDT'
stop_loss = 0.05
All_Data = {}
sym_3h = {}

# wait_until_divisible_by_3()

All_Data[symbol] = Read_Currency(exchange, symbol, 36)
sym_3h[symbol] = []
for i in range(0, 32, 3):
    sym_3h[symbol].append(Mojesa_Class_Func(All_Data[symbol].iloc[i:i + 3]))

sym_df = pd.DataFrame(sym_3h[symbol])
sym_df = add_t_s_c(sym_df)
sym_df

,Date,O-Open,O-Close,O-High,O-Low,O-Volume,O-Bull,C-Open,C-High,C-Close,...,T-Close,T-Low,T-Mean,C-Mean,D-Mean,brate,vrate,Trend,State,Color
0,2024-08-12 00:00:00,58712.59,58717.25,59088.33,58112.25,1494.702249,white,58712.59,59088.33,58454.01,...,58717.25,58112.25,58585.630,58583.300,58652.630,2.000774,0.333581,NaN,NaN,NaN
1,2024-08-12 03:00:00,58717.25,58571.56,58818.50,58363.09,1005.268204,black,58717.25,58818.50,58509.50,...,58717.25,58363.09,58613.375,58613.375,58516.660,14.507682,0.401619,NaN,NaN,NaN
2,2024-08-12 06:00:00,58571.56,58317.42,59025.66,57650.00,1280.213586,black,58411.18,58973.33,58674.62,...,58674.62,57650.00,58496.020,58542.900,58496.020,0.737514,0.392387,1.0,3.0,violet
3,2024-08-12 09:00:00,58317.42,59851.70,59970.00,58125.75,2374.033487,white,58296.68,58994.98,58969.60,...,59851.70,58125.75,59074.190,58633.140,59410.650,0.762861,0.383670,5.0,1.0,none
4,2024-08-12 12:00:00,59851.70,60292.53,60700.12,57805.25,2707.766944,white,58138.30,60700.12,60292.53,...,60292.53,57805.25,59215.415,59215.415,59789.960,17.445983,0.328903,3.0,1.0,none
5,2024-08-12 15:00:00,60292.53,59292.55,60510.24,59249.28,1991.737241,black,60292.53,60510.24,59620.00,...,60292.53,59249.28,59792.540,59956.265,59456.275,2.053840,0.409471,4.0,6.0,blue
6,2024-08-12 18:00:00,59292.55,58841.96,59490.01,58444.01,1795.138594,black,58524.00,59389.48,59032.55,...,59292.55,58444.01,58908.275,58778.275,58908.275,0.661701,0.360450,0.0,5.0,none
7,2024-08-12 21:00:00,58841.96,59346.64,59745.98,58555.26,1149.806818,white,58841.96,59745.98,59346.64,...,59346.64,58555.26,59094.300,59094.300,59010.745,6.165913,0.426202,4.0,6.0,blue
8,2024-08-13 00:00:00,59346.64,59481.99,59928.71,59269.85,1159.775989,white,59346.64,59890.62,59801.01,...,59801.01,59269.85,59573.825,59573.825,59770.505,7.447468,0.403761,6.0,2.0,blue
9,2024-08-13 03:00:00,59481.99,59211.99,59520.00,58831.04,1335.746312,black,59481.99,59520.00,59040.00,...,59481.99,58831.04,59260.995,59260.995,59215.995,55.179775,0.372670,0.0,5.0,none


In [7]:
All_Data[symbol]

,timestamp,Open,High,Low,Close,Volume
Date,,,,,,
2024-08-12 00:00:00,1723420800000,58712.59,59088.33,58112.25,58454.01,720.163569
2024-08-12 01:00:00,1723424400000,58454.01,58796.01,58278.46,58588.01,400.654892
2024-08-12 02:00:00,1723428000000,58588.01,58760.45,58319.27,58717.25,373.883788
2024-08-12 03:00:00,1723431600000,58717.25,58818.50,58443.76,58509.50,359.618139
2024-08-12 04:00:00,1723435200000,58509.50,58675.56,58363.09,58523.82,288.048716
2024-08-12 05:00:00,1723438800000,58523.82,58698.26,58398.88,58571.56,357.601349
2024-08-12 06:00:00,1723442400000,58571.56,58638.00,58144.04,58411.18,374.472557
2024-08-12 07:00:00,1723446000000,58411.18,58973.33,57650.00,58674.62,544.965256
2024-08-12 08:00:00,1723449600000,58674.62,59025.66,58256.90,58317.42,360.775773


In [ ]:

current_trade = None

while True:
    current_price = exchange.fetch_ticker(symbol)['last']

    data = find_last_three_colors(sym_df)

    if current_trade is None:

        [state, position] = check_open_conditions(data)

        if state == 'open':
            entry_time = datetime.utcnow()
            n_entry_time = position[0]
            trade_type = position[2]
            position_type = position[1]
            current_trade = Trade(entry_time, current_price, trade_type, position, cash=10000, commission_rate=0.001)
            log_open(current_trade, position_type, n_entry_time)

        else:
            wait_until_divisible_by_3()
            new_data = fetch_data(exchange, symbol, since=exchange.parse8601(sym_df['timestamp'].max().strftime('%Y-%m-%dT%H:%M:%S')))
            sym_3h[symbol].append(Mojesa_Class_Func(new_data))
            sym_df = pd.DataFrame(sym_3h[symbol])
            sym_df = add_t_s_c(sym_df)[sym_df]

    elif current_trade:
        status = check_stop_loss(current_price, stop_loss)
        if status == 'close':
            current_trade = None
        last_data = {}
        last_data[symbol] = Read_Currency(exchange, symbol, 1 * 60, 3)
        sym_3h[symbol].append(Mojesa_Class_Func(last_data[symbol]))
        sym_df = pd.DataFrame(sym_3h[symbol])
        sym_df = add_t_s_c(sym_df)[sym_df]
        status = check_close_condition(current_trade, sym_df, stop_loss, current_price)
        if status == 'close':
            current_trade = None





